# filter the samples resolved by forward tracing and tracking failure samples

In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn')
import matplotlib
import numpy as np
import cv2,os
import joblib
store_dir='./Velocity_Pos/'
all_result=np.load('./unprocess2.npy')

if not os.path.exists(store_dir):
      os.makedirs(store_dir) 
for id in all_result:
    V_dict=joblib.load('./Motion/'+str(id)+'.pkl')
    
    v_list=V_dict['velocity']
    
    #import pdb; pdb.set_trace()
    if np.max(v_list)>12:
        if len(v_list)>145 or len(v_list)<30:
            if len(v_list)<30:
                r_dict[id]=105
                print(id)
            print(id)
            #print(id)
            plt.figure(figsize=(14, 10))
            sns.lineplot(data=v_list)
            plt.ylim([0,250])
            plt.savefig(store_dir+str(id)+'.jpg')


# peak supression

In [ ]:
def supress(v_list):
    v_list=np.array(v_list)
    tops=np.argsort(-v_list)
    for idx in tops[0:7:2]:
        neighbor=10-idx
        if v_list[idx]>12  and (v_list[idx]>v_list[idx-neighbor:idx]).all() \
            and (v_list[idx]>v_list[idx+1:idx+neighbor]).all():
            v_list[idx]=0

    return v_list

# Detection Mechanism

In [1]:
from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error
from sklearn.metrics import median_absolute_error, mean_squared_error, mean_squared_log_error

def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
def plotMovingAverage(series,id_v, window, plot_intervals=False, scale=2.5, plot_anomalies=False):


    rolling_mean = series.rolling(window=window).mean()
    
    plt.figure(figsize=(15,5))
    plt.title("Moving average\n window size = {}".format(window))
    plt.plot(rolling_mean, "g", label="Rolling mean trend")

    # Plot confidence intervals for smoothed values
    if plot_intervals:
        mae = mean_absolute_error(series[window:], rolling_mean[window:])
        deviation = np.std(series[window:] - rolling_mean[window:])
        lower_bond = rolling_mean - (mae + scale * deviation)
        upper_bond = rolling_mean + (mae + scale * deviation)
        plt.plot(upper_bond, "r--", label="Upper Bond / Lower Bond")
        plt.plot(lower_bond, "r--")
        
        # Having the intervals, find abnormal values
        if plot_anomalies:
            anomalies = pd.DataFrame(index=series.index, columns=series.columns)
            #anomalies[series<lower_bond] = series[series<lower_bond]
            anomalies[series>upper_bond] = series[series>upper_bond]
            
            plt.plot(anomalies, "ro", markersize=10)
            if len(anomalies):
                np.save(store_dir+str(id)+'.npy',anomalies)
        
    plt.plot(series[window:], label="Actual values")
    plt.legend(loc="upper left")
    plt.grid(True)
    plt.savefig(store_dir+str(id)+'.jpg')

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn')
import matplotlib
import numpy as np
import cv2,os
import joblib
store_dir='./Angle_Pos/'
all_result=np.load('./unprocess2.npy')
if not os.path.exists(store_dir):
      os.makedirs(store_dir) 
for id in all_result:
 if id in [11,35,45,139,72,99,104,105,23,43,119,134,50,92]:
    V_dict=joblib.load('./Motion/'+str(id)+'.pkl')
    #plt.figure(figsize=(14, 10))
    v_list=V_dict['velocity']
    a=0.8
    v_list=supress(v_list)
    v_list=[(1-a)*v_list[i-1]+a*v_list[i] for i,v in enumerate(v_list) if i!=0]
    
    series=pd.DataFrame(data=v_list)
    
    
    plotMovingAverage(series,id,window=5,plot_intervals=True,plot_anomalies=True)

# Post-processing

In [ ]:
store_dir='./Angle_Pos/'
all_result=np.load('./unprocess2.npy')
r_dict={}
for id in [11,35,72,99,104,105,23,43,119,134,50,92]:
    b=np.load(store_dir+str(id)+'.npy',allow_pickle=True)
    
    a = [a_ for a_ in b if a_ == a_]
    
    if len(a):
        max_index=np.argmax(a)
        max_value=np.max(a)
        left_than_max=a[:max_index]
        local_min=0
        for i,v in enumerate(left_than_max):
            if i ==0:
                local_min=0
            elif i !=0:
                if v>left_than_max[i-1]:
                    continue
                else:
                    local_min=i
                    break
        if local_min==0:
            if len(a[max_index:])>=2:
                index=np.where(b==a[-1])
        
                r_dict[id]=int(index[0])-7
            else:
                print(id,'Has Acc!')
        else:
            index=np.where( b== max_value)
            r_dict[id]=int(index[0])
            print(id,"Accident found",int(index[0]))
            
    else:
        print(id,'No Anomaly')